"""
This Script is the workflow of bed inversion in SermeQ-OGGM-PyGEM
@Authors: Ruitang revised based on the processing steps in OGGM and PyGEM
Date: 18-Oct-2024
"""




In [ ]:
source activate pygem_env

In [31]:
# import libs
import os
import numpy as np
import sys
import traceback
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np

# Local libraries (PyGEM)
try:
    import pygem
except:
    sys.path.append(os.getcwd() + '/../PyGEM/')
import pygem_input as pygem_prms
import pygem.pygem_modelsetup as modelsetup
from pygem.massbalance import PyGEMMassBalance # https://github.com/Ruitangtang/PyGEM/blob/80571ec215491276f97db74ee2a95682290d1132/pygem/massbalance.py#L17
from pygem.oggm_compat import single_flowline_glacier_directory_with_calving # https://github.com/Ruitangtang/PyGEM/blob/80571ec215491276f97db74ee2a95682290d1132/pygem/oggm_compat.py#L115
from pygem.shop import debris 
from pygem import class_climate

# Local libraries (OGGM)
import oggm
oggm_version = float(oggm.__version__[0:3])
from oggm import cfg, tasks, utils
from oggm.core import gis, flowline, inversion, massbalance
from oggm.core.calving_Jan_Ruitang import CalvingFluxBasedModelJanRt # https://github.com/Ruitangtang/oggm/blob/4bfe248ef9564c6c727236e4b40d0f814ffe0874/oggm/core/calving_Jan_Ruitang.py#L406
from oggm.core.inversion_RT_New import find_inversion_calving_from_any_mb # https://github.com/Ruitangtang/oggm/blob/4bfe248ef9564c6c727236e4b40d0f814ffe0874/oggm/core/inversion_RT_New.py#L1720
if oggm_version > 1.301:
    from oggm.core.massbalance import apparent_mb_from_any_mb # Newer Version of OGGM
else:
    from oggm.core.climate import apparent_mb_from_any_mb # Older Version of OGGM






In [15]:
#pwd

In [18]:
# set up the tempory working directory
pygem_prms.main_directory = '/home/ruitang/OGGM-Ruitang/Results/Test_KS_1T_24Jun/RGI_17.15808_Test/'
cfg.PATHS['working_dir'] = '/home/ruitang/OGGM-Ruitang/Results/Test_KS_1T_24Jun/RGI_17.15808_Test/oggm_gdirs/'
# download the gdir
glacier_rgi_id = '17.15808'
try:
    gdir = single_flowline_glacier_directory_with_calving(glacier_rgi_id, 
                                                    logging_level='DEBUG',
                                                    reset=True)
except:
    print('Error in downloading the glacier')
    print(traceback.format_exc())

2024-10-18 11:55:32: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-10-18 11:55:32: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-10-18 11:55:32: oggm.cfg: Multiprocessing: using all available processors (N=32)
2024-10-18 11:55:32: oggm.cfg: PARAMS['border'] changed from `80` to `160`.
2024-10-18 11:55:32: oggm.cfg: PARAMS['dl_verify'] changed from `False` to `True`.
2024-10-18 11:55:32: oggm.cfg: PARAMS['use_multiple_flowlines'] changed from `True` to `False`.
2024-10-18 11:55:32: urllib3.connectionpool: Starting new HTTPS connection (1): cluster.klima.uni-bremen.de:443
2024-10-18 11:55:32: urllib3.connectionpool: https://cluster.klima.uni-bremen.de:443 "GET /~oggm/gdirs/oggm_v1.6/L1-L2_files/2023.2/elev_bands_w_data/RGI62/b_160/L2/ HTTP/1.1" 200 2605
2024-10-18 11:55:32: oggm.workflow: init_glacier_directories from prepro level 2 on 1 glaciers.
2024-10-18 11:55:32: oggm.workflow: Execute entity tasks [gdir_from_

the path of 'mb_obs'in gdir is : /home/ruitang/OGGM-Ruitang/Results/Test_KS_1T_24Jun/RGI_17.15808/oggm_gdirs/per_glacier/RGI60-17/RGI60-17.15/RGI60-17.15808/mb_data.pkl


In [19]:
gdir.is_tidewater = True
cfg.PARAMS['use_kcalving_for_inversion'] = True
cfg.PARAMS['use_kcalving_for_run'] = True

2024-10-18 11:56:07: oggm.cfg: PARAMS['use_kcalving_for_run'] changed from `False` to `True`.


In [20]:
fls = gdir.read_pickle('inversion_flowlines')
glacier_area = fls[0].widths_m * fls[0].dx_meter
debris.debris_binned(gdir, fl_str='inversion_flowlines', ignore_debris=True)

2024-10-18 11:56:43: pygem.shop.debris: (RGI60-17.15808) debris_binned


In [23]:
main_glac_rgi_all = modelsetup.selectglaciersrgitable(glac_no=['17.15808'])
# Tidewater glaciers
termtype_list = [1,5]
main_glac_rgi = main_glac_rgi_all.loc[main_glac_rgi_all['TermType'].isin(termtype_list)]
main_glac_rgi.reset_index(inplace=True, drop=True)

glc_no ['17.15808']
rfp ['17_rgi60_SouthernAndes.csv', '14_rgi60_SouthAsiaWest.csv', '19_rgi60_AntarcticSubantarctic.csv', '08_rgi60_Scandinavia.csv', '13_rgi60_CentralAsia.csv', '06_rgi60_Iceland.csv', '07_rgi60_Svalbard.csv', '09_rgi60_RussianArctic.csv', '18_rgi60_NewZealand.csv', '01_rgi60_Alaska.csv', '12_rgi60_CaucasusMiddleEast.csv', '10_rgi60_NorthAsia.csv', '02_rgi60_WesternCanadaUS.csv', '04_rgi60_ArcticCanadaSouth.csv', '16_rgi60_LowLatitudes.csv', '05_rgi60_GreenlandPeriphery.csv', '11_rgi60_CentralEurope.csv', '15_rgi60_SouthAsiaEast.csv', '03_rgi60_ArcticCanadaNorth.csv']
rgi_glc_num all
17
rgi_glac_number: ['15808']
regs: 17_rgi60_SouthernAndes.csv
regs: 14_rgi60_SouthAsiaWest.csv
regs: 19_rgi60_AntarcticSubantarctic.csv
regs: 08_rgi60_Scandinavia.csv
regs: 13_rgi60_CentralAsia.csv
regs: 06_rgi60_Iceland.csv
regs: 07_rgi60_Svalbard.csv
regs: 09_rgi60_RussianArctic.csv
regs: 18_rgi60_NewZealand.csv
regs: 01_rgi60_Alaska.csv
regs: 12_rgi60_CaucasusMiddleEast.csv
regs: 10_r

In [26]:


main_glac_rgi_all = modelsetup.selectglaciersrgitable(glac_no=['17.15808'])
# Tidewater glaciers
termtype_list = [1,5]
main_glac_rgi = main_glac_rgi_all.loc[main_glac_rgi_all['TermType'].isin(termtype_list)]
main_glac_rgi.reset_index(inplace=True, drop=True)

# ===== TIME PERIOD =====
dates_table = modelsetup.datesmodelrun(
        startyear=pygem_prms.ref_startyear, endyear=pygem_prms.ref_endyear, spinupyears=pygem_prms.ref_spinupyears,
        option_wateryear=pygem_prms.ref_wateryear)
# print('dates_table is :',dates_table)
# ===== LOAD CLIMATE DATA =====
# Climate class
assert pygem_prms.ref_gcm_name in ['ERA5', 'ERA-Interim'], (
        'Error: Calibration not set up for ' + pygem_prms.ref_gcm_name)
gcm = class_climate.GCM(name=pygem_prms.ref_gcm_name)
# Air temperature [degC]
gcm_temp, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.temp_fn, gcm.temp_vn, main_glac_rgi, dates_table)
if pygem_prms.option_ablation == 2 and pygem_prms.ref_gcm_name in ['ERA5']:
    gcm_tempstd, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.tempstd_fn, gcm.tempstd_vn,
                                                                    main_glac_rgi, dates_table)
else:
    gcm_tempstd = np.zeros(gcm_temp.shape)
# Precipitation [m]
gcm_prec, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.prec_fn, gcm.prec_vn, main_glac_rgi, dates_table)
# Elevation [m asl]
gcm_elev = gcm.importGCMfxnearestneighbor_xarray(gcm.elev_fn, gcm.elev_vn, main_glac_rgi)
# Lapse rate [degC m-1]
gcm_lr, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.lr_fn, gcm.lr_vn, main_glac_rgi, dates_table)
# Add climate data to glacier directory
nglac = 0
gdir.historical_climate = {'elev': gcm_elev[nglac],
                            'temp': gcm_temp[nglac,:],
                            'tempstd': gcm_tempstd[nglac,:],
                            'prec': gcm_prec[nglac,:],
                            'lr': gcm_lr[nglac,:]}
gdir.dates_table = dates_table

glc_no ['17.15808']
rfp ['17_rgi60_SouthernAndes.csv', '14_rgi60_SouthAsiaWest.csv', '19_rgi60_AntarcticSubantarctic.csv', '08_rgi60_Scandinavia.csv', '13_rgi60_CentralAsia.csv', '06_rgi60_Iceland.csv', '07_rgi60_Svalbard.csv', '09_rgi60_RussianArctic.csv', '18_rgi60_NewZealand.csv', '01_rgi60_Alaska.csv', '12_rgi60_CaucasusMiddleEast.csv', '10_rgi60_NorthAsia.csv', '02_rgi60_WesternCanadaUS.csv', '04_rgi60_ArcticCanadaSouth.csv', '16_rgi60_LowLatitudes.csv', '05_rgi60_GreenlandPeriphery.csv', '11_rgi60_CentralEurope.csv', '15_rgi60_SouthAsiaEast.csv', '03_rgi60_ArcticCanadaNorth.csv']
rgi_glc_num all
17
rgi_glac_number: ['15808']
regs: 17_rgi60_SouthernAndes.csv
regs: 14_rgi60_SouthAsiaWest.csv
regs: 19_rgi60_AntarcticSubantarctic.csv
regs: 08_rgi60_Scandinavia.csv
regs: 13_rgi60_CentralAsia.csv
regs: 06_rgi60_Iceland.csv
regs: 07_rgi60_Svalbard.csv
regs: 09_rgi60_RussianArctic.csv
regs: 18_rgi60_NewZealand.csv
regs: 01_rgi60_Alaska.csv
regs: 12_rgi60_CaucasusMiddleEast.csv
regs: 10_r

In [29]:
#%% Invert ice thickness
# ---- Model parameters ------
kp_value = None
tbias_value = None
# Use most likely parameters from initial calibration to force the mass balance gradient for the inversion
# Use the calibrated model parameters (although they were calibrated without accounting for calving)

modelprms_fn = glacier_rgi_id + '-modelprms_dict.pkl'
modelprms_fp = (pygem_prms.output_filepath + 'calibration/' + glacier_rgi_id.split('.')[0].zfill(2) 
                + '/')
assert os.path.exists(modelprms_fp + modelprms_fn), 'modelprms_dict file does not exist'
with open(modelprms_fp + modelprms_fn, 'rb') as f:
    modelprms_dict = pickle.load(f)
modelprms_em = modelprms_dict['emulator']
kp_value = modelprms_em['kp'][0]
#print("the initial kp_value from the emulator is:",kp_value)
tbias_value = modelprms_em['tbias'][0]
#print("the initial tbias value from the emulator is:",tbias_value)
    
# Otherwise use input parameters
if kp_value is None:
    kp_value = pygem_prms.kp
if tbias_value is None:
    tbias_value = pygem_prms.tbias

# Set model parameters
modelprms = {'kp': kp_value,
                'tbias': tbias_value,
                'ddfsnow': pygem_prms.ddfsnow,
                'ddfice': pygem_prms.ddfice,
                'tsnow_threshold': pygem_prms.tsnow_threshold,
                'precgrad': pygem_prms.precgrad}                
    
# Calving and dynamic parameters
calving_k = 1.5
# calving_k is the calving parameter in k_calving or Sermeq, we use the same notation just for easy transfer from different calving models, 
# they have the same magnitude range but with different uinit, which will be processed in the calving law it self, for the SermeQ, we set it ranges (0.1,3)
cfg.PARAMS['calving_k'] = calving_k
cfg.PARAMS['inversion_calving_k'] = cfg.PARAMS['calving_k']



2024-10-18 13:28:23: oggm.cfg: PARAMS['calving_k'] changed from `0.6` to `1.5`.
2024-10-18 13:28:23: oggm.cfg: PARAMS['inversion_calving_k'] changed from `0.6` to `1.5`.


In [38]:
# Select subsets of data
glacier_rgi_table = main_glac_rgi.loc[main_glac_rgi.index.values[nglac], :]
glacier_str = '{0:0.5f}'.format(glacier_rgi_table['RGIId_float'])
# glen_a and fs

# Here, we read the glen_a and fs from the glena_reg_fullfn, which is the regional calibration of the glen_a and fs (get from David Rounce), 
# for each RGI region, it has its own 'glens_a_multiplier' and fs, However, somehow the fs = 0, always, so we read the 'glens_a_multiplier' from the regional result,
#  but hard coded the fs as 

# if pygem_prms.use_reg_glena:
#     glena_df = pd.read_csv(pygem_prms.glena_reg_fullfn)
#     glena_idx = np.where(glena_df.O1Region == main_glac_rgi.O1Region)[0][0]
#     glen_a_multiplier = glena_df.loc[glena_idx,'glens_a_multiplier'] # here for this 17 region is 5.330457913310021,so we hard coded here 
#     fs = glena_df.loc[glena_idx,'fs'] # here in the regional calibration, the fs is always 0, so we hard coded here
# else:
#     fs = pygem_prms.fs
#     glen_a_multiplier = pygem_prms.glen_a_multiplier


# hard coded fs
glen_a_multiplier = 5.330457913310021 
fs = 5.7e-20

In [40]:
# CFL number (may use different values for calving to prevent errors)

if not glacier_rgi_table['TermType'] in [1,5] or not pygem_prms.include_calving:
    cfg.PARAMS['cfl_number'] = pygem_prms.cfl_number
else:
    cfg.PARAMS['cfl_number'] = pygem_prms.cfl_number_calving

2024-10-18 13:42:19: oggm.cfg: PARAMS['cfl_number'] changed from `0.02` to `0.01`.


In [41]:
# ----- Mass balance model for ice thickness inversion using OGGM -----
mbmod_inv = PyGEMMassBalance(gdir, modelprms, glacier_rgi_table,
                                hindcast=pygem_prms.hindcast,
                                debug=pygem_prms.debug_mb,
                                debug_refreeze=pygem_prms.debug_refreeze,
                                fls=fls, option_areaconstant=True,
                                inversion_filter=False)
# ----- CALVING -----
# Number of years (for OGGM's run_until_and_store)
if pygem_prms.timestep == 'monthly':
    nyears = int(dates_table.shape[0]/12)
else:
    assert True==False, 'Adjust nyears for non-monthly timestep'
print("nyears is (int(dates_table.shape[0]/12)) :",nyears)
mb_years=np.arange(nyears)
try:
    print("invert_standard is False & The find_inversion_calving_from_any_mb start")
    print("⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅")
    out_calving = find_inversion_calving_from_any_mb(gdir, mb_model= mbmod_inv, mb_years=mb_years,
                                                    glen_a=cfg.PARAMS['glen_a']*glen_a_multiplier, fs=fs,calving_law_inv = None,
                                                    modelprms = modelprms, glacier_rgi_table = glacier_rgi_table,
                                                    hindcast=pygem_prms.hindcast,debug=pygem_prms.debug_mb,
                                                    debug_refreeze=pygem_prms.debug_refreeze,option_areaconstant=True,
                                                    inversion_filter=False)
    print("The find_inversion_calving_from_any_mb end")
    print("the out claving is:",out_calving)
except:
    print("Something wrong with the find_inversion_calving_from_any_mb")
    print(traceback.format_exc())


2024-10-18 13:46:05: oggm.core.inversion_RT_New: (RGI60-17.15808) find_inversion_calving_from_any_mb




DEBUGGING MASS BALANCE FUNCTION


nyears is (int(dates_table.shape[0]/12)) : 20
invert_standard is False & The find_inversion_calving_from_any_mb start
⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅
inversion calving rate is 0
cmb is (calving mass loss with unit mm a-1): 0.0
is calving: False
mb_years: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
**************** get_annual_mb end ****************
*****

2024-10-18 13:46:06: oggm.core.inversion_RT_New: (RGI60-17.15808) thickness, freeboard before water and frontal ablation: 94.54813131430511, 115.37434998618835


=========== the  0 iteration in mass_conservation_inversion ===========
***************** min_rel_h is <=1 *****************
shape_factor in _compute_thick is : [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
========== Start to recompute thickness where parabola is too flat ========== 
k in the iterations of mass_conservation_inversion now is: 1
=========== the  1 iteration in mass_conservation_inversion ===========
***************** min_rel_h is <=1 *****************
shape_factor in _compute_thick is : [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

2024-10-18 13:46:14: oggm.core.inversion_RT_New: (RGI60-17.15808) find_inversion_calving_from_any_mb: found calving flux of 0.003 km3 yr-1


The inversion calving k is: 1.5
the thick in inversion is 94.54813131430511
the water depth is : [200.76693749]
the free board is : 39.37434998618835
the thick in inversion now is [240.14128748]
the flux is (km3 a-1); [0.14811595] based on the calving law is: None  ; flux in k_clving (km3 a-1) is : [0.14811595]
shape_factor in _compute_thick is : 1.0
The inversion calving k is: 1.5
the thick in inversion is 94.54813131430511
the water depth is : [120.80025056]
the free board is : 39.37434998618835
the thick in inversion now is [160.17460054]
the flux is (km3 a-1); [0.05944349] based on the calving law is: None  ; flux in k_clving (km3 a-1) is : [0.05944349]
shape_factor in _compute_thick is : 1.0
The inversion calving k is: 1.5
the thick in inversion is 94.54813131430511
the water depth is : [120.80025068]
the free board is : 39.37434998618835
the thick in inversion now is [160.17460066]
the flux is (km3 a-1); [0.05944349] based on the calving law is: None  ; flux in k_clving (km3 a-1)

2024-10-18 13:46:15: oggm.core.inversion_RT_New: (RGI60-17.15808) found frontal thickness, water depth, freeboard, water level of 54.67567279996387, 17.30132281377552, 37.37434998618835, 78.0
2024-10-18 13:46:15: oggm.core.inversion_RT_New: (RGI60-17.15808) calving (law) flux of 0.0029061381377740326 (0.002906138137774229)


k in the iterations of mass_conservation_inversion now is: 3
--------------------------------------------------------
h_tol is : 0.1
np.any(h_diff > h_tol) is : False
--------------------------------------------------------
The inversion calving k is: 1.5
the thick in inversion is 54.67567279996283
the water depth is : 17.30132281377552
the free board is : 37.37434998618835
the thick in inversion now is 54.67567279996387
the flux is (km3 a-1); 0.002906138137774229 based on the calving law is: None  ; flux in k_clving (km3 a-1) is : 0.002906138137774229
The find_inversion_calving_from_any_mb end
the out claving is: {'calving_flux': 0.0029061381377740326, 'calving_rate_myr': 25.951984220661522, 'calving_law_flux': 0.002906138137774229, 'calving_water_level': 78.0, 'calving_inversion_k': 1.5, 'calving_front_slope': 0.15087418545311146, 'calving_front_water_depth': 17.30132281377552, 'calving_front_free_board': 37.37434998618835, 'calving_front_thick': 54.67567279996387, 'calving_front_wid

In [42]:
# ------ MODEL WITH EVOLVING AREA ------
tasks.init_present_time_glacier(gdir) # adds bins below
debris.debris_binned(gdir, fl_str='model_flowlines')  # add debris enhancement factors to flowlines
nfls = gdir.read_pickle('model_flowlines')
# Mass balance model
mbmod = PyGEMMassBalance(gdir, modelprms, glacier_rgi_table,
                            hindcast=pygem_prms.hindcast,
                            debug=pygem_prms.debug_mb,
                            debug_refreeze=pygem_prms.debug_refreeze,
                            fls=nfls, option_areaconstant=False)
# Water Level
# Check that water level is within given bounds
cls = gdir.read_pickle('inversion_output')[-1]
th = cls['hgt'][-1]
thick0 = cls['thick'][-1]
rho = cfg.PARAMS['ice_density']
rho_o = cfg.PARAMS['ocean_density'] # Ocean density, must be >= ice density
water_level = out_calving ['calving_water_level']
# "------------------ after the thickness inversion with calving, run the dynamics ------------------")
#%%
ev_model = CalvingFluxBasedModelJanRt(nfls, y0=0, mb_model=mbmod,
                            glen_a=cfg.PARAMS['glen_a']*glen_a_multiplier, fs=fs,
                            is_tidewater=gdir.is_tidewater,
                            water_level=water_level
                            )
# Run model
try:
    # add the condition for different situation,1. do_fl_diag = True 2. do_fl_diag = False
    do_fl_diag = cfg.PARAMS['store_fl_diagnostics']
    if do_fl_diag:
        fl_diag_path = gdir.get_filepath('fl_diagnostics',delete=True)
        diag, fl_diag_dss= ev_model.run_until_and_store(nyears,store_monthly_step= True,fl_diag_path=fl_diag_path)
        print('diag is :',diag)
    else:
        diag = ev_model.run_until_and_store(nyears,store_monthly_step= True)
        print('diag is :',diag)
except:
    print("something is wrong with the run_until_and_store")
    print(traceback.format_exc())



# print the result
print("the volume_3 in the diag is :",diag.volume_m3)
ev_model.mb_model.glac_wide_volume_annual[-1] = diag.volume_m3[-1]
ev_model.mb_model.glac_wide_area_annual[-1] = diag.area_m2[-1]
calving_m3_month = (diag.calving_m3.values[1:] - diag.calving_m3.values[0:-1]) 
print(calving_m3_month.shape[0],len(ev_model.mb_model.glac_wide_frontalablation))
for n in np.arange(calving_m3_month.shape[0]):
    ev_model.mb_model.glac_wide_frontalablation[n] = calving_m3_month[n]*pygem_prms.density_ice / pygem_prms.density_water

# Glacier-wide total mass balance (m3 w.e.)
ev_model.mb_model.glac_wide_massbaltotal = (
        ev_model.mb_model.glac_wide_massbaltotal  - ev_model.mb_model.glac_wide_frontalablation)

# Output of calving
out_calving_forward = {}
# calving flux (km3 ice/yr)
out_calving_forward['calving_flux'] = calving_m3_month.sum() / nyears / 1e9
# calving flux (Gt/yr)
#calving_flux_Gta = out_calving_forward['calving_flux'] * pygem_prms.density_ice / pygem_prms.density_water
calving_flux_Gta = out_calving_forward['calving_flux'] *1e9* pygem_prms.density_ice / 1e12   

2024-10-18 13:54:54: oggm.core.flowline: (RGI60-17.15808) init_present_time_glacier
2024-10-18 13:54:54: pygem.shop.debris: (RGI60-17.15808) debris_binned
2024-10-18 13:54:54: oggm.cfg: WARNING: adding an unknown parameter `max_calving_stretch_distance`:`8000` to PARAMS.




DEBUGGING MASS BALANCE FUNCTION


self.yr is : 0.0
y0 is : 0.0
y1 is : 20
monthly_time is: [ 0.          0.08333333  0.16666667  0.25        0.33333333  0.41666667
  0.5         0.58333333  0.66666667  0.75        0.83333333  0.91666667
  1.          1.08333333  1.16666667  1.25        1.33333333  1.41666667
  1.5         1.58333333  1.66666667  1.75        1.83333333  1.91666667
  2.          2.08333333  2.16666667  2.25        2.33333333  2.41666667
  2.5         2.58333333  2.66666667  2.75        2.83333333  2.91666667
  3.          3.08333333  3.16666667  3.25        3.33333333  3.41666667
  3.5         3.58333333  3.66666667  3.75        3.83333333  3.91666667
  4.          4.08333333  4.16666667  4.25        4.33333333  4.41666667
  4.5         4.58333333  4.66666667  4.75        4.83333333  4.91666667
  5.          5.08333333  5.16666667  5.25        5.33333333  5.41666667
  5.5         5.58333333  5.66666667  5.75        5.83333333  5.91666667
  6.          6.08333333  6.166